# Day 16: Packet Decoder

Leading zeros should be preserved.

In [1]:
def parse(input):
    """Parses bit string from input."""
    return ''.join(format(int(char, base=16), '04b') for char in input.strip())

assert parse('38006F45291200') == '00111000000000000110111101000101001010010001001000000000'

In [2]:
def decode(packet):
    """Returns packet values"""
    # Tracks location in packet.
    head = 0
    
    # Version is 3 bits.
    version = int(packet[head:(head := head + 3)], base=2)
    
    # Type is 3 bits.
    type = int(packet[head:(head := head + 3)], base=2)
    
    if type != 4:
        # It's an operator packet containing one or more subpackets.
        subpackets = []
        
        # Length type is 1 bit.
        length_type = int(packet[head:(head := head + 1)])
        
        if length_type:
            # next 11 bits are a number that represents the number of sub-packets 
            # immediately contained by this packet.
            length = int(packet[head:(head := head + 11)], base=2)
            for _ in range(length):
                subpacket = decode(packet[head:])
                head += subpacket['head']
                subpackets.append(subpacket)
            return dict(version=version, type=type, length_type=length_type, subpackets=subpackets, head=head)

        else:
            # next 15 bits are a number that represents the total length in bits 
            # of the sub-packets contained by this packet..
            length = int(packet[head:(head := head + 15)], base=2)

            # Keep extracting subpackets until we reach length.
            while length > 0:
                subpacket = decode(packet[head:])
                length -= subpacket['head']
                head += subpacket['head'] 
                subpackets.append(subpacket)
            return dict(version=version, type=type, length_type=length_type, subpackets=subpackets, head=head)


    else:
        # Type 4 is a literal value.
        # Stitch together 5-bit chunks until MSB is 0.
        value_chunks = []
        while True:
            # 1-bit flag indicates if more chunks follow.
            more = int(packet[head:(head := head + 1)], base=2)
            
            # Append the 4-bit *string*.
            value_chunks.append(packet[head:(head := head + 4)])
            
            if not more:
                # Consume any remaining bits to align to 4-bit hex boundary.
                return dict(version=version, type=type, value=int(''.join(value_chunks), base=2), head=head)

Literal value 2021.

In [3]:
assert decode(parse('D2FE28'))['value'] == 2021

This operator packet should have two literal subpackets: 10 and 20.

In [4]:
decode(parse('38006F45291200'))

{'version': 1,
 'type': 6,
 'length_type': 0,
 'subpackets': [{'version': 6, 'type': 4, 'value': 10, 'head': 11},
  {'version': 2, 'type': 4, 'value': 20, 'head': 16}],
 'head': 49}

This operator packet should have three literal subpackets: 1, 2 and 3.

In [5]:
decode(parse('EE00D40C823060'))

{'version': 7,
 'type': 3,
 'length_type': 1,
 'subpackets': [{'version': 2, 'type': 4, 'value': 1, 'head': 11},
  {'version': 4, 'type': 4, 'value': 2, 'head': 11},
  {'version': 1, 'type': 4, 'value': 3, 'head': 11}],
 'head': 51}

In [6]:
def walk_versions(packet):
    """Yields versions in packet and subpackets."""
    yield packet['version']
    for packet in packet.get('subpackets', []):
        yield from walk_versions(packet)

This packet has a version sum of 16.

In [7]:
sum(walk_versions(decode(parse('8A004A801A8002F478'))))

16

This packet has a version sum of 12.

In [8]:
sum(walk_versions(decode(parse('620080001611562C8802118E34'))))

12

This packet has a version sum of 23.

In [9]:
sum(walk_versions(decode(parse('C0015000016115A2E0802F182340'))))

23

This packet has a version sum of 31.

In [10]:
sum(walk_versions(decode(parse('A0016C880162017C3686B18A3D4780'))))

31

This packet has a version sum of 16.

In [11]:
sum(walk_versions(decode(parse(open('day-16-input.txt').read()))))

895

# Part two

In [12]:
import math

def compute(packet):
    """Computes packet according to operators."""
    # Literal values (type ID 4) represent a single number
    if packet['type'] == 4:
        return packet['value']
    # Packets with type ID 0 are sum packets - their value is the sum of the values of their sub-packets. 
    # If they only have a single sub-packet, their value is the value of the sub-packet.
    elif packet['type'] == 0:
        return sum(compute(packet) for packet in packet['subpackets'])
    # Packets with type ID 1 are product packets - their value is the result of multiplying together the values of their sub-packets. 
    # If they only have a single sub-packet, their value is the value of the sub-packet.
    elif packet['type'] == 1:
        return math.prod(compute(packet) for packet in packet['subpackets'])
    # Packets with type ID 2 are minimum packets - their value is the minimum of the values of their sub-packets.
    elif packet['type'] == 2:
        return min(compute(packet) for packet in packet['subpackets'])
    # Packets with type ID 3 are maximum packets - their value is the maximum of the values of their sub-packets.
    elif packet['type'] == 3:
        return max(compute(packet) for packet in packet['subpackets'])
    # Packets with type ID 5 are greater than packets - their value is 1 if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
    elif packet['type'] == 5:
        first, second = packet['subpackets']
        return int(compute(first) > compute(second))
    # Packets with type ID 6 are less than packets - their value is 1 if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
    elif packet['type'] == 6:
        first, second = packet['subpackets']
        return int(compute(first) < compute(second))
    # Packets with type ID 7 are equal to packets - their value is 1 if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
    elif packet['type'] == 7:
        first, second = packet['subpackets']
        return int(compute(first) == compute(second))

C200B40A82 finds the sum of 1 and 2, resulting in the value 3.

In [13]:
compute(decode(parse('C200B40A82')))

3

04005AC33890 finds the product of 6 and 9, resulting in the value 54.

In [14]:
compute(decode(parse('04005AC33890')))

54

880086C3E88112 finds the minimum of 7, 8, and 9, resulting in the value 7.

In [15]:
compute(decode(parse('880086C3E88112')))

7

CE00C43D881120 finds the maximum of 7, 8, and 9, resulting in the value 9.

In [16]:
compute(decode(parse('CE00C43D881120')))

9

D8005AC2A8F0 produces 1, because 5 is less than 15.

In [17]:
compute(decode(parse('D8005AC2A8F0')))

1

F600BC2D8F produces 0, because 5 is not greater than 15.

In [18]:
compute(decode(parse('F600BC2D8F')))

0

9C005AC2F8F0 produces 0, because 5 is not equal to 15.

In [19]:
compute(decode(parse('9C005AC2F8F0')))

0

9C0141080250320F1802104A08 produces 1, because 1 + 3 = 2 * 2.

In [20]:
compute(decode(parse('9C0141080250320F1802104A08')))

1

Evaluate input.

In [21]:
compute(decode(parse(open('day-16-input.txt').read())))

1148595959144